<a href="https://colab.research.google.com/github/edurso/tfod-wkspc/blob/master/notebooks/test_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /tf/models /tf/workspace
!mkdir -p /tf/data
from google.colab import drive
import os
drive.mount('/tf/data')

Mounted at /tf/data


In [2]:
# install deps 
!python --version
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim pycocotools

Python 3.7.12
     |████████████████████████████████| 492.1 MB 15 kB/s 
     |████████████████████████████████| 1.4 MB 38.6 MB/s 
     |████████████████████████████████| 462 kB 51.1 MB/s 
     |████████████████████████████████| 2.1 MB 40.2 MB/s 
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.22.0
    Uninstalling tensorflow-io-gcs-filesystem-0.22.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.22.0
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 352 kB 5.2 MB/s 


In [3]:
# clear previous runs
!mkdir -p /tf/models/ /tf/workspace/

# clone repos
!git clone --depth 1 https://github.com/tensorflow/models /tf/models
!git clone --depth 1 https://github.com/edurso/tfod-wkspc /tf/workspace

Cloning into '/tf/models'...
remote: Enumerating objects: 3159, done.
remote: Counting objects: 100% (3159/3159), done.
remote: Compressing objects: 100% (2689/2689), done.
remote: Total 3159 (delta 810), reused 1328 (delta 426), pack-reused 0
Receiving objects: 100% (3159/3159), 33.40 MiB | 26.88 MiB/s, done.
Resolving deltas: 100% (810/810), done.
Cloning into '/tf/workspace'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 42 (delta 7), reused 18 (delta 2), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [ ]:
# compile protobufs
%cd /tf/models/research/
!protoc object_detection/protos/*.proto --python_out=.

# install object detector package
!cp /tf/models/research/object_detection/packages/tf2/setup.py . 
!pip install .

/tf/models/research
Processing /tf/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.9 MB 8.4 MB/s 
     |████████████████████████████████| 1.8 MB 38.2 MB/s 
     |████████████████████████████████| 23.1 MB 1.3 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 43.7 MB/s 
     |████████████████████████████████| 1.2 MB 45.0 MB/s 
     |████████████████████████████████| 47.7 MB 95 kB/s 
     |████████████████████████████████| 4.9 MB 38.2 MB/s 
     |████████████████████████████████| 213 kB 53.5 MB/s 
     |█████████████████████████████

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import tensorflow as tf
print(tf.__version__)

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [ ]:
PREFIX = '/tf/data/MyDrive/tensorflow/power-port-targeting/'
PATH_TO_MODEL = PREFIX + 'exported-models/trained_model/saved_model'
PATH_TO_LABELS = PREFIX + 'annotations/label_map.pbtxt'
PATH_TO_TEST_IMAGES_DIR = PREFIX + 'images/test'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
detection_model = load_model(model_name)
print(detection_model.signatures['serving_default'].inputs)

def load_model(location):
    
    model = tf.saved_model.load(str(location))
    return model

def infer(model, image):

    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]

    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
    if 'detection_masks' in output_dict:
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                output_dict['detection_masks'], output_dict['detection_boxes'],
                image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict

def show_inference(model, image_path):

    image_np = np.array(Image.open(image_path))

    output_dict = infer(model, image_np)

    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=8)

    display(Image.fromarray(image_np))

In [ ]:
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob('*.jpg')))
for image_path in TEST_IMAGE_PATHS:
    show_inference(detection_model, image_path)

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS)
IMAGE_PATH = TEST_IMAGE_PATHJS[0]

img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()